In [1]:
from datetime import datetime
from data_loader import dataloader
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
import pickle
from sklearn.metrics import mean_squared_error
import pandas as pd
import csv

In [2]:
user,ratings,movies = dataloader()
print(user.__len__)
print(movies.tail())
print(ratings.head())

<bound method DataFrame.__len__ of       user_id gender  age  occupation    zip
0           1      F    1          10  48067
1           2      M   56          16  70072
2           3      M   25          15  55117
3           4      M   45           7  02460
4           5      M   25          20  55455
...       ...    ...  ...         ...    ...
6035     6036      F   25          15  32603
6036     6037      F   45           1  76006
6037     6038      F   56           1  14706
6038     6039      F   45           0  01060
6039     6040      M   25           6  11106

[6040 rows x 5 columns]>
      movie_id                       title          genres
3878      3948     Meet the Parents (2000)          Comedy
3879      3949  Requiem for a Dream (2000)           Drama
3880      3950            Tigerland (2000)           Drama
3881      3951     Two Family House (2000)           Drama
3882      3952       Contender, The (2000)  Drama|Thriller
   user_id  movie_id  rating  timestamp
0    

In [4]:
movie_index_by_id = {id: i for i, id in enumerate(movies["movie_id"])}
# print(movie_index_by_id)
genres = [
    "Action",
    "Adventure",
    "Animation",
    "Children's",
    "Comedy",
    "Crime",
    "Documentary",
    "Drama",
    "Fantasy",
    "Film-Noir",
    "Horror",
    "Musical",
    "Mystery",
    "Romance",
    "Sci-Fi",
    "Thriller",
    "War",
    "Western",
]
genre_index_by_name = {name:i for i, name in enumerate(genres)}
print(genre_index_by_name)
print(type(movie_index_by_id[1]))


{'Action': 0, 'Adventure': 1, 'Animation': 2, "Children's": 3, 'Comedy': 4, 'Crime': 5, 'Documentary': 6, 'Drama': 7, 'Fantasy': 8, 'Film-Noir': 9, 'Horror': 10, 'Musical': 11, 'Mystery': 12, 'Romance': 13, 'Sci-Fi': 14, 'Thriller': 15, 'War': 16, 'Western': 17}
<class 'int'>


In [4]:
import numpy as np
# build binary array for movie genres
movie_features = np.zeros((len(movies), len(genres)))
for i, movie_genres in enumerate(movies["genres"]):
    for genre in movie_genres.split("|"):        
        genre_index = genre_index_by_name[genre]
        movie_features[i, genre_index] = 1
movie_features

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

In [5]:
def get_weight(user_id):
    user_ratings = ratings[ratings['user_id']==user_id]
    rating = user_ratings['rating']
    movie_indexes = [movie_index_by_id[movie_id] for movie_id in user_ratings["movie_id"]]
    movie_data = movie_features[movie_indexes]
    movie_data = np.transpose(movie_data)
    c = movie_data @ rating
    for i in range(len(c)):
        if sum(movie_data[i]) > 2 :
            c[i] =float (c[i]/sum(movie_data[i]))
        else:
            c[i] = c[i]/2
    return c
user_model_dict = {}
for user_id in user["user_id"].unique():
    user_model_dict[user_id] = get_weight(user_id)


In [6]:
import math
dict_u  = []
for i in range(len(user_model_dict)):
    for j in range(len(user_model_dict[i+1])):
        if(not(math.isnan(user_model_dict[i+1][j])) and user_model_dict[i+1][j] !=0 ):
            dict_u.append({'user_id':i+1, 'genres':j+3883,'weight':int(user_model_dict[i+1][j]) })
print(dict_u[0])

{'user_id': 1, 'genres': 3883, 'weight': 4}


In [7]:
dict = ["user_id", "genres", "weight"]
p = pd.DataFrame(columns=dict, data=dict_u )
print(p)

       user_id  genres  weight
0            1    3883       4
1            1    3884       4
2            1    3885       4
3            1    3886       4
4            1    3887       4
...        ...     ...     ...
93882     6040    3896       3
93883     6040    3897       3
93884     6040    3898       3
93885     6040    3899       3
93886     6040    3900       4

[93887 rows x 3 columns]


In [8]:
p.to_csv(r'u_dict.csv', index= None)